In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [2]:
class RunBuilder():
    @staticmethod#static? iets met dat je m kan callen using the class itself. don't need an instance of the class, to call the method. i guess dat je m dus niet eerst hoeft te initieren alszijnde type x? 
    def get_runs(params):
        
        Run = namedtuple('Run',params.keys())#die ordereddicttionary heeft keys and values. dit heb je wel eens eerder gezien I guess.
                                             #blijkbaar maakt ie een mooie string als je die keys zo oproept. test dat even.
        
        runs = []
        for v in product(*params.values()): #dit doet dus iets dat ie per value combo nieuwe iteratie doet
            runs.append(Run(*v))
            
        return runs

In [3]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None

    def begin_run(self,run,network,loader): 
        
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images,labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('images',grid)
        self.tb.add_graph(self.network,images)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    def end_epoch(self):
        
        epoch_duration = time.time()-self.epoch_start_time
        run_duration = time.time()-self.run_start_time
        
        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        
        self.tb.add_scalar('Loss',loss,self.epoch_count)
        self.tb.add_scalar('Accuracy',accuracy,self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f'{name}.grad',param.grad,self.epoch_count)
            
        results = OrderedDict()
        results["run"]=self.run_count
        results["epoch"]=self.epoch_count
        results["loss"]=loss
        results["accuracy"]=accuracy
        results["epoch duration"]=epoch_duration
        results["run duration"]=run_duration
                            
        for k,v in self.run_params._asdict().items():  #deze komen uit run, je batch_size & lr
            results[k] = v #geloof dat je hier dus voor elke run met andere batch size etc. maar 1 lr en batchsize toevoegt, vandaar dat dit niet in de loop zit
        self.run_data.append(results) #1 batch_size en lr bij de results bij, en vervolgens voeg je al je results toe aan wat je metadata i guess
        df = pd.DataFrame.from_dict(self.run_data,orient='columns') #dit zorgt dat het in een leuk tabelletje staat
                                
        clear_output(wait=True)
        display(df)
                                
    def track_loss(self,loss):
            self.epoch_loss += loss.item()* self.loader.batch_size
                                    
    def track_num_correct(self,preds,labels):
            self.epoch_num_correct += self._get_num_correct(preds,labels)
        
    @torch.no_grad()
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self,fileName):
        
        pd.DataFrame.from_dict(
        self.run_data
        ,orient = 'columns'
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json','w',encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [4]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t,dim=1)

        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()
                                                                                         ])
)

In [6]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000,10000]
)
m=RunManager()
for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set,batch_size=run.batch_size)
    optimiser = optim.Adam(network.parameters(),lr=run.lr)
    
    m.begin_run(run,network,loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            
            images,labels=batch
            preds = network(images) #pass batch
            loss = F.cross_entropy(preds,labels) # calculate loss
            optimiser.zero_grad() # Zero gradients
            loss.backward() # calculate gradients
            optimiser.step() # update weights
            
            m.track_loss(loss)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size
0,1,1,0.972673,0.628900,11.747690,13.776542,0.010,1000
1,1,2,0.522765,0.798033,11.759553,25.658768,0.010,1000
2,1,3,0.436353,0.840333,14.410464,40.184922,0.010,1000
3,1,4,0.390896,0.856100,13.688395,54.103702,0.010,1000
4,1,5,0.356462,0.868500,15.222293,69.446673,0.010,1000
5,2,1,2.136714,0.224183,14.831318,23.230689,0.010,10000
6,2,2,1.398761,0.474400,14.804958,38.160618,0.010,10000
7,2,3,1.042370,0.594783,21.872275,60.172518,0.010,10000
8,2,4,0.873290,0.653650,23.514763,83.962874,0.010,10000
9,2,5,0.782270,0.688317,15.339552,99.465989,0.010,10000
